In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
dataLocation = './games.csv'
data = pd.read_csv(dataLocation)

In [3]:
data

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2
20054,WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,A41,Queen's Pawn,2
20055,yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,D00,Queen's Pawn Game: Mason Attack,3
20056,b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,B07,Pirc Defense,4


## Preprocessing

In [ ]:
data['winner'].unique()

array(['white', 'black', 'draw'], dtype=object)

## We are going to predict only on the basis of moves of the players

In [ ]:
dataWithoutDraw = data.query("winner != 'draw'")
# separately deal with moves
dataWithoutDraw

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2
20054,WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,A41,Queen's Pawn,2
20055,yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,D00,Queen's Pawn Game: Mason Attack,3
20056,b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,B07,Pirc Defense,4


In [ ]:
moves = dataWithoutDraw['moves']
labels = dataWithoutDraw['winner'].apply(lambda x : 0 if x == 'black' else 1 if x == 'white' else 2)

## We have converted labels to numeric values

In [ ]:
labels = np.array(labels)

In [ ]:
labels.shape

(19108,)

## Unique number of moves across all game

In [ ]:
all_moves = set()

for move_list in moves :
  for move in move_list.split(' ') :
    if move not in all_moves : 
      all_moves.add(move)

max_vocab = len(all_moves)

In [ ]:
max_vocab

4373

## Length of longest sequence of move

In [ ]:
max_len = 0;

for move_list in moves :
  total = 0
  for move in move_list.split(' ') :
    total += 1;
  max_len = max(max_len, total)



In [ ]:
max_len

349

In [ ]:
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(moves)

# numerical equivalent of moves
sequences = tokenizer.texts_to_sequences(moves)

In [ ]:
# word_index is a dictionary that maps a move to a number
word_index = tokenizer.word_index

word_index

In [ ]:
# adding zeroes to the begining of the moves lists which have length less than max_len
model_inputs = pad_sequences(sequences, maxlen=max_len)
model_inputs.shape

(19108, 349)

## **Now the data is ready to be feed to a neural network**

### The last two values in every row are white ratings and black ratings respectively

In [ ]:
model_inputs[0: 3]

array([[  0,   0,   0, ...,   9, 363,  58],
       [  0,   0,   0, ...,  41,  11,  60],
       [  0,   0,   0, ..., 202, 521, 563]], dtype=int32)

# **Training**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(model_inputs, labels, train_size=0.7, random_state=24)

## **Model**

In [ ]:
embedding_dim = 256

inputs = tf.keras.Input(shape=max_len)

# embedding layer booasts up a particular number into higher dimension ( here 256 )
# embedding layer because there is a critically deep connection between previous move and current move
# we can't show that connection if the move is represented by a single number
# we are actually representing a single move as a vector

embedding = tf.keras.layers.Embedding(
    input_dim = max_vocab,
    output_dim = embedding_dim,
    input_length = max_len + 2
)(inputs)

gru = tf.keras.layers.GRU(units=embedding_dim)(embedding)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(gru)

In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


In [ ]:
batch_size = 32
epochs = 3

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau()],
    verbose=2
)

Epoch 1/3
335/335 - 485s - loss: 0.5497 - accuracy: 0.7218 - auc: 0.7960 - val_loss: 0.5047 - val_accuracy: 0.7604 - val_auc: 0.8369 - lr: 0.0010 - 485s/epoch - 1s/step
Epoch 2/3
335/335 - 478s - loss: 0.4283 - accuracy: 0.7978 - auc: 0.8837 - val_loss: 0.3376 - val_accuracy: 0.8564 - val_auc: 0.9303 - lr: 0.0010 - 478s/epoch - 1s/step
Epoch 3/3
335/335 - 478s - loss: 0.2990 - accuracy: 0.8750 - auc: 0.9451 - val_loss: 0.2928 - val_accuracy: 0.8789 - val_auc: 0.9467 - lr: 0.0010 - 478s/epoch - 1s/step


In [ ]:
model.evaluate(X_test, y_test)

180/180 [==============================] - 58s 322ms/step - loss: 0.3165 - accuracy: 0.8632 - auc: 0.9391


[0.3165043294429779, 0.8632478713989258, 0.9390978813171387]